In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


### Table with results

In [2]:
TEST_SPLIT = .1
N_FOLDS=10
path="results_with_scaling_default"

In [3]:
cols=["nb","knn","svm", "gsvm","logreg","rf"]
tuples=[
    ('counts', ''),
    ('fasttext', ''),
 ('G', 'post'),
 ('G', 'pre'),
 ('G', 'long'),
 ('M', 'post'),
 ('M', 'pre'),
 ('M', 'long')]
ind=pd.MultiIndex.from_tuples(tuples, names=['feat', 'seg'])

results_df=pd.DataFrame(columns=cols,index=ind)


In [4]:

for classif in cols:
    for feat in ["fasttext","counts","G","M"]:
        if feat in ["fasttext","counts"]:
            file = open(path+os.sep+"f1s_folds_"+feat+"_"+classif+".out",'r') 
            file_text=file.read()
            print(file_text)
            f1=float(file_text.split('\n')[0].split(', ')[1])
            results_df[classif][feat]=f1 if f1>0.3 else "/"

            continue
        else:
            for seg in ["post", "pre","long"]:
                if seg=='long':
                    file = open(path+os.sep+"f1s_folds_"+seg+"_"+feat+"_"+classif+".out",'r')  
                else:
                    file = open(path+os.sep+"f1s_folds_"+feat+"_"+seg+"_"+classif+".out",'r') 
                file_text=file.read()
                results_df[classif][(feat,seg)]=float(file_text.split('\n')[0].split(', ')[1])


f1, 0.6679233114337955
acc, 0.6826475056736594

f1, 0.6208057907704625
acc, 0.7050376223294019

f1, 0.7424078000984682
acc, 0.7512788939868682

f1, 0.10105480001800837
acc, 0.13705978466732166

f1, 0.7978729917418415
acc, 0.8026764706996694

f1, 0.7236399480174311
acc, 0.7317052961101704

f1, 0.8029823383617988
acc, 0.8076402123217866

f1, 0.03256917626666803
acc, 0.16441312952438245

f1, 0.7953545265902513
acc, 0.8021857323093595

f1, 0.7616515567440015
acc, 0.7710169638229145

f1, 0.740601560093301
acc, 0.7588683370021104

f1, 0.6337651843202889
acc, 0.6840134462887258



In [28]:
tuples=[('BoW', 'full text'),
    ('Fasttext', 'full text'),
 ('BERT-German', 'first segment'),
 ('BERT-German', 'last segment'),
 ('BERT-German', 'all segments'),
 ('BERT-Multilingual', 'first segment'),
 ('BERT-Multilingual', 'last segment'),
 ('BERT-Multilingual', 'all segments')]
ind=pd.MultiIndex.from_tuples(tuples, names=['features', 'segments'])
results_df.columns=["NB","KNN","SVM-Linear","SVM-RBF","LogReg","RF"]
results_df.index=ind

In [33]:
results_df
#results_df.to_csv("../visualizations/results_df.csv", sep=',', header=True, index=True)

NB       KNN SVM-Linear   SVM-RBF  \
features          segments                                                 
BoW               full text      0.620806         /    0.72364         /   
Fasttext          full text      0.667923  0.742408   0.797873  0.802982   
BERT-German       first segment  0.634293   0.73415   0.806747  0.815175   
                  last segment   0.613016  0.739374   0.804345  0.808784   
                  all segments   0.659497   0.74005   0.810932  0.814622   
BERT-Multilingual first segment  0.531304  0.701701   0.797708   0.79533   
                  last segment   0.540752  0.709477   0.786417    0.7872   
                  all segments   0.603151  0.713843   0.795655  0.796184   

                                   LogReg        RF  
features          segments                           
BoW               full text      0.761652  0.633765  
Fasttext          full text      0.795355  0.740602  
BERT-German       first segment   0.81384  0.732333  
                  last segment   0.804329  0.731649  
                  all segments   0.805548  0.732781  
BERT-Multilingual first segment  0.782503  0.700148  
                  last segment   0.780492   0.70258  
                  all segments   0.781035  0.701722

### Confusion Matrix

In [33]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#y_labels=pd.read_csv("y_labels.out", index_col=0, header=None) 

#labels=y_labels.iloc[:,0].unique()
c="gsvm"
settings='G_post'
labels= ['Sport', 'Kultur', 'Web', 'Wirtschaft', 'Inland', 'Etat', 'International', 'Panorama', 'Wissenschaft']
conf_mat=np.genfromtxt(path+os.sep+"conf_mat_folds_"+settings+"_"+c+".out", delimiter=",")
#conf_mat=np.int_(conf_mat)
df_cm = pd.DataFrame(conf_mat, index=labels, columns=labels)
sn.set(font_scale=1.3)
df_cm=df_cm.astype(int)
plt.figure(figsize = (9,7))
ax=sn.heatmap(df_cm, annot=True, fmt='g',annot_kws={"size": 13.5}, cmap="YlGnBu") # font size
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
figure = ax.get_figure()    
figure.savefig('../visualizations/conf_mat.png', dpi=500,bbox_inches='tight')

### Classification report

In [56]:
#y_labels=pd.read_csv("y_labels.out", index_col=0, header=None) 
labels=['Sport', 'Kultur', 'Web', 'Wirtschaft', 'Inland', 'Etat', 'International', 'Panorama', 'Wissenschaft']
c="gsvm"
settings='G_post'
conf_mat=np.genfromtxt(path+os.sep+"conf_mat_folds_"+settings+"_"+c+".out", delimiter=",")
cols=["Precision","Recall", "F1 Score"]
metrics_df=pd.DataFrame(index=labels, columns=cols)
true_pos = np.diag(conf_mat)
supp=np.sum(conf_mat, axis=0)
false_pos = supp - true_pos
false_neg = np.sum(conf_mat, axis=1) - true_pos
prec = true_pos / (true_pos + false_pos)
rec = true_pos / (true_pos + false_neg)
f1=2*prec*rec/(prec+rec)
metrics_df["Precision"]=prec
metrics_df["Recall"]=rec
metrics_df["F1 Score"]=f1
metrics_df["Support"]=supp
averaged=[np.mean(prec), np.mean(rec), np.mean(f1)]
temp=pd.DataFrame(data=np.array(averaged).reshape(1,3), index=["Overall"], columns=cols)
metrics_df=metrics_df.append(temp)
metrics_df["Support"]["Overall"]=""
metrics_df.to_csv("../visualizations/final_metrics.csv", sep=',', header=True, index=True)

In [57]:
metrics_df

,F1 Score,Precision,Recall,Support
Sport,0.973496,0.983844,0.963364,1176
Kultur,0.794007,0.801512,0.786642,529
Web,0.882088,0.887681,0.876565,1656
Wirtschaft,0.779743,0.763587,0.796598,1472
Inland,0.722484,0.691269,0.756650,1111
Etat,0.763432,0.822107,0.712575,579
International,0.827201,0.821265,0.833223,1533
Panorama,0.761763,0.761310,0.762217,1680
Wissenschaft,0.845045,0.873371,0.818499,537
Overall,0.816584,0.822883,0.811815,
